# Make your requests faster

When you start scraping web-pages or requesting APIs, you will be facing a problem when doing a lot of requests: this is really slow!

It's because python is slow! You could say, well it should not be. Let's see how we can speed things up!

## Measure performances

In this notebook we will need to track how much time some code is taking to execute.
To make things easier, we will create a simple decorator that will print the number of micro-seconds a function takes to execute.

A good opportonity to practice decorators in a practical example!

*Note that you need python3.3 or higher.*

In [1]:
import time

def print_timing(func):
    '''Create a timing decorator function use @print_timing just above the function you want to time.'''

    def wrapper(*arg):
        start = time.perf_counter()
        
        # Run the function decorated
        result = func(*arg)

        end = time.perf_counter()
        execution_time = round((end - start), 2)
        print(f'{func.__name__} took {execution_time} sec')
        return result

    return wrapper


@print_timing
def example():
    time.sleep(4)


example()

example took 4.0 sec


## The API

For this example, we will use the [quotable.io](https://api.quotable.io) API. It's an online API you can use to generate random quote.

But feel free to replace `api_url` value with any API you'd like.

In [3]:
api_url = "https://api.quotable.io/random"

## The "classic" way

If you start playing with requests, your should probably have something like this:

In [4]:
import requests


def basic_request(url: str):
    response = requests.get(url)
    response_json = response.json()
    print(response_json["content"])


@print_timing
def basic_loop_request(url: str):
    # Query 50 times the API
    for _ in range(50):
        basic_request(url)


basic_loop_request(api_url)

Dreams pass into the reality of action. From the actions stems the dream again; and this interdependence produces the highest form of living.
To have much learning and skill, to be well-trained in discipline, and good in speech — this is the highest blessing.
Friendship is held to be the severest test of character. It is easy, we think, to be loyal to a family and clan, whose blood is in your own veins.
The eye sees only what the mind is prepared to comprehend.
What is past is left behind. The future is yet unreached. Whatever quality is present, clearly see it right there.
Go to your bosom: Knock there and ask your heart what it doth know.
The awareness of our own strength makes us modest.
Trust your own instinct. Your mistakes might as well be your own, instead of someone else's.
Allow the world to live as it chooses, and allow yourself to live as you choose.
Do not wait for leaders; do it alone, person to person.
All children are artists. The problem is how to remain an artist once 

### Results

On my machine it took **17.06 sec for 50 requests**. 

Pretty slow right? But why is that?

Each time you make a request, your computer needs to create a new "session", format your request, send it and wait to receive the response before doing it again with the next request.

## The "session" way

To speed this, we can use a **"session"** that will be share by all the requests.

You can picture it as a postman that knows you already, so he knows which bell to ring, where is the mailbox,... Instead of having to search for those each time.

In [5]:
from requests import Session


def session_request(url: str, session: Session):
    # Instead of using request.get, we use our session
    response = session.get(url)
    response_json = response.json()
    print(response_json["content"])


@print_timing
def session_loop_request(url: str):
    # Create shared session for all of your requests
    with Session() as session:
        # Query 50 times the API
        for _ in range(50):
            session_request(url, session)


session_loop_request(api_url)

Blessed is the man who expects nothing, for he shall never be disappointed.
They must often change, who would be constant in happiness or wisdom.
The superior man understands what is right; the inferior man understands what will sell.
Mistakes are always forgivable, if one has the courage to admit them.
Believe deep down in your heart that you're destined to do great things.
The real problem is not whether machines think but whether men do.
He who learns must suffer. And even in our sleep pain that cannot forget falls drop by drop upon the heart, and in our own despair, against our will, comes wisdom to us by the awful grace of God.
We respect our elders. There is wisdom that comes from experience, and I am not going to stop learning from wise counsel.
Beauty is not in the face; beauty is a light in the heart.
The best cure for the body is a quiet mind.
Write your plans in pencil and give God the eraser.
If you're going through hell, keep going.
Wisdom ceases to be wisdom when it becom

### Results

It took me **5.99 sec for 50 requests**. That's better!

And as you can see, I didn't change that much in the code.

## The "Async" way

If you need even more performances, you will need to use [AsyncIo](https://docs.python.org/3/library/asyncio.html).

This is a library to allow you to run asynchronous code.

Why is that more efficiant? Well, when you send a request you need to wait for the response. And during the waiting time, our computer does nothing.
If you count all the time the computer is just "waiting" on 50 or more requests, you will be surprised to see that most of the computing time is just waiting for the server to respond.

[AsyncIo](https://docs.python.org/3/library/asyncio.html) allow you to bypass that.

But as always, it has a cost: complexity.

Making your code async will complixify the code a lot and make the debugging not a pleasant experience. Also, you will go so fast that you could be banned by the server.

My advice? Use it only if you need it.

I will show you a simple example but you want to understand it better, I really advice you **[this video](https://www.youtube.com/watch?v=qAh5dDODJ5k)**!

### Requirements
In order to simplify a bit the code, I will use [httpx](https://www.python-httpx.org/) a python library that is working the same way as the `requests` module but with few helpers for async.

In [6]:
%pip install httpx

### Warning!
This code won't work in jupyter notebook, there are subtilities for async in jupyter notebook. See [this thread](https://stackoverflow.com/questions/47518874/how-do-i-run-python-asyncio-code-in-a-jupyter-notebook) for more informations.

To make it simpler, I will put this code in a .py file and run it in command line:

```python
from httpx import AsyncClient
import asyncio
import time


api_url = "https://api.quotable.io/random"


async def session_request_async(url: str, session: AsyncClient):
    # Instead of using request.get, we use our session
    response = await session.get(url)
    response_json = response.json()
    print(response_json["content"])
    return response_json


async def session_loop_request_async(url: str):
    # Create shared session for all of your requests
    async with AsyncClient() as session:
        # Create a list of empty tasks
        tasks = []
        # Query 50 times the API
        for _ in range(50):
            # Add a request to tasks
            tasks.append(
                asyncio.create_task(
                    session_request_async(url, session)        
                )
            )
        # Now that all the tasks are registred, run them
        responses = await asyncio.gather(*tasks)
            
            


start = time.perf_counter()

# We need to use asyncio.run to run the async function
asyncio.run(session_loop_request_async(api_url))

end = time.perf_counter()
execution_time = round((end - start), 2)
print(f'session_loop_request_async took {execution_time} sec')
```

In [10]:
!python ./assets/async_requests.py

Courage is not the absence of fear, but simply moving on with dignity despite that fear.
Be slow to fall into friendship; but when thou art in, continue firm and constant.
All difficult things have their origin in that which is easy, and great things in that which is small.
I'll prepare and someday my chance will come.
Learning is the beginning of wealth. Learning is the beginning of health. Learning is the beginning of spirituality. Searching and learning is where the miracle process all begins.
I have not failed. I've just found 10,000 ways that won't work.
Chance is always powerful. Let your hook be always cast; in the pool where you least expect it, there will be a fish.
We have committed the Golden Rule to memory; let us now commit it to life.
Technology… is a queer thing. It brings you great gifts with one hand, and it stabs you in the back with the other.
Love is the only force capable of transforming an enemy into a friend.
We are wiser than we know.
You cannot step twice into 

### Results
It only took me **0.8 sec for 50 requests**! That's impressive.

But as you can see, it is harder to write, structure and debug. So make sure you **really** need it if you consider using this method.

## Summary

If we gather all our results:

| Method                     | Execution time for 50 requests |
|----------------------------|--------------------------------|
| `requests.get` loop        | 17.06 sec                  |
| `requests` with `Session`  | 5.99 sec                   |
| `httpx` with `AsyncClient` | 0.8 sec                   |